In [ ]:
import os
import numpy as np
import librosa
import librosa.display  # Added for spectrogram visualization
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import seaborn as sns

The dataset was automatically get splited as Training and  Testing dataset as 20% from  environment_audio and gunshots_audio

In [ ]:
# ========== SETTINGS ==========
DATASET_PATH = r"dataset"  # Path to the dataset directory
SAMPLE_RATE = 16000
DURATION = 2
SAMPLES_PER_CLIP = SAMPLE_RATE * DURATION
MAX_PAD_LEN = 44

# ========== DATASET SIZE CONTROL ==========
# Set to None to use all available files, or specify a number to limit the dataset size
NUM_ENVIRONMENT_FILES = NUM_GUNSHOT_FILES = 2000 # Using smaller dataset for demo/export

print(f"Dataset Configuration:")
print(f"Environment files to use: {NUM_ENVIRONMENT_FILES if NUM_ENVIRONMENT_FILES else 'ALL'}")
print(f"Gunshot files to use: {NUM_GUNSHOT_FILES if NUM_GUNSHOT_FILES else 'ALL'}")
print(f"Sample rate: {SAMPLE_RATE} Hz")
print(f"Audio duration: {DURATION} seconds")
print(f"MFCC features: 13 coefficients x {MAX_PAD_LEN} time frames")

In [ ]:
# ========== CHECK AVAILABLE FILES ==========
def check_available_files():
    env_path = os.path.join(DATASET_PATH, "environment_audio")
    gun_path = os.path.join(DATASET_PATH, "gunshots_audio")
    
    env_files = [f for f in os.listdir(env_path) if f.endswith(".wav")]
    gun_files = [f for f in os.listdir(gun_path) if f.endswith(".wav")]
    
    print(" Available Audio Files:")
    print(f"   Environment files: {len(env_files)}")
    print(f"   Gunshot files: {len(gun_files)}")
    print(f"   Total available: {len(env_files) + len(gun_files)}")

    return len(env_files), len(gun_files)

# Run the check
available_env, available_gun = check_available_files()

In [ ]:
# ========== FEATURE EXTRACTION ==========
def extract_features(file_path):
    try:
        audio, sr = librosa.load(file_path, sr=SAMPLE_RATE)
        if len(audio) < SAMPLES_PER_CLIP:
            audio = np.pad(audio, (0, SAMPLES_PER_CLIP - len(audio)))
        else:
            audio = audio[:SAMPLES_PER_CLIP]
        mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
        if mfccs.shape[1] < MAX_PAD_LEN:
            pad_width = MAX_PAD_LEN - mfccs.shape[1]
            mfccs = np.pad(mfccs, ((0, 0), (0, pad_width)), mode='constant')
        else:
            mfccs = mfccs[:, :MAX_PAD_LEN]
        return mfccs.T  # shape (44, 13)
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

In [ ]:
# ========== LOAD DATA ==========
def load_and_extract():
    X_env, X_gun = [], []
    
    # Get environment audio files
    env_files = [f for f in os.listdir(os.path.join(DATASET_PATH, "environment_audio")) if f.endswith(".wav")]
    if NUM_ENVIRONMENT_FILES is not None:
        env_files = env_files[:NUM_ENVIRONMENT_FILES]
    
    print(f"Processing {len(env_files)} environment audio files...")
    for i, file in enumerate(env_files):
        f = extract_features(os.path.join(DATASET_PATH, "environment_audio", file))
        if f is not None:
            X_env.append(f)
        if (i + 1) % 50 == 0:  # Progress indicator
            print(f"  Processed {i + 1}/{len(env_files)} environment files")
    
    # Get gunshot audio files
    gun_files = [f for f in os.listdir(os.path.join(DATASET_PATH, "gunshots_audio")) if f.endswith(".wav")]
    if NUM_GUNSHOT_FILES is not None:
        gun_files = gun_files[:NUM_GUNSHOT_FILES]
    
    print(f"Processing {len(gun_files)} gunshot audio files...")
    for i, file in enumerate(gun_files):
        f = extract_features(os.path.join(DATASET_PATH, "gunshots_audio", file))
        if f is not None:
            X_gun.append(f)
        if (i + 1) % 50 == 0:  # Progress indicator
            print(f"  Processed {i + 1}/{len(gun_files)} gunshot files")

    print(f"\nDataset loaded:")
    print(f"Environment samples: {len(X_env)}")
    print(f"Gunshot samples: {len(X_gun)}")
    print(f"Total samples: {len(X_env) + len(X_gun)}")
    
    return np.array(X_env), np.array(X_gun)

In [ ]:
# ========== MAIN ==========
print("Loading and processing data...")
X_env, X_gun = load_and_extract()

print(f"\nBalancing dataset:")
print(f"   Original environment samples: {len(X_env)}")
print(f"   Original gunshot samples: {len(X_gun)}")

# Balance the dataset - resample environment data to match gunshot data size
if len(X_env) != len(X_gun):
    if len(X_env) > len(X_gun):
        X_env = resample(X_env, n_samples=len(X_gun), random_state=42)
        print(f"   ↳ Resampled environment data to {len(X_gun)} samples")
    else:
        X_gun = resample(X_gun, n_samples=len(X_env), random_state=42)
        print(f"   ↳ Resampled gunshot data to {len(X_env)} samples")

# Combine datasets
X = np.concatenate((X_env, X_gun))
y = np.array([0]*len(X_env) + [1]*len(X_gun))

print(f"\nFinal balanced dataset:")
print(f"   Environment samples (class 0): {len(X_env)}")
print(f"   Gunshot samples (class 1): {len(X_gun)}")
print(f"   Total samples: {len(X)}")
print(f"   Feature shape per sample: {X_env[0].shape}")

# Add channel dimension for CNN
X = np.expand_dims(X, -1)  # shape: (samples, 44, 13, 1)
print(f"   Final input shape: {X.shape}")
print(f"   Labels shape: {y.shape}")
print("Data preparation completed!")

In [ ]:
# ========== SPLIT ==========
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# ========== MODEL ==========
model = models.Sequential([
    layers.Input(shape=(44, 13, 1)),
    layers.Conv2D(8, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(4, (3, 3), activation='relu'),
    layers.GlobalAveragePooling2D(),
    layers.Dense(8, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# ========== TRAIN ==========
print("Training model...")
history = model.fit(X_train, y_train, epochs=10, batch_size=8, validation_split=0.1, verbose=0)

# ========== EVALUATE ==========
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {acc*100:.2f}%")
print(f"Test Loss: {loss*100:.2f}")

# ========== PREDICTIONS FOR CONFUSION MATRIX ==========
y_pred_probs = model.predict(X_test, verbose=0)
y_pred = (y_pred_probs > 0.5).astype(int).flatten()

In [ ]:
# 1. Model Accuracy Graph
train_acc = history.history['accuracy']  # should have length 10
val_acc = history.history['val_accuracy']  # should have length 10
plt.figure(figsize=(8, 6))
epochs = range(len(train_acc))
plt.plot(epochs, train_acc, color='steelblue', label='Train Accuracy', linewidth=2)
plt.plot(epochs, val_acc, color='orange', label='Validation Accuracy', linewidth=2)
plt.title('Model Accuracy', fontsize=14)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)
plt.legend(loc='upper left', fontsize=11, frameon=False)
plt.grid(True, alpha=0.3)
plt.ylim([0.62, 1])
plt.tight_layout()
# Fix the filename formatting
filename_suffix = f"_{NUM_GUNSHOT_FILES}" if NUM_GUNSHOT_FILES else "_full"
plt.savefig(f'images/accuracy_plot_{filename_suffix}.png', dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
# 2. Model Loss Graph  
train_loss = history.history['loss']  # This should be a list of floats, one per epoch
val_loss = history.history['val_loss']  # Same here
plt.figure(figsize=(8, 6))
epochs = range(len(train_loss))  # Replace train_loss with your actual loss array
plt.plot(epochs, train_loss, color='steelblue', label='Train Loss', linewidth=2)
plt.plot(epochs, val_loss, color='orange', label='Validation Loss', linewidth=2)
plt.title('Model Loss', fontsize=14)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.legend(loc='upper right', fontsize=11, frameon=False)
plt.grid(True, alpha=0.3)
plt.ylim([min(min(train_loss), min(val_loss)) * 0.9, max(max(train_loss), max(val_loss)) * 1.1])  # Dynamic limit for better zoom
plt.tight_layout()
# Fix the filename formatting
filename_suffix = f"_{NUM_GUNSHOT_FILES}" if NUM_GUNSHOT_FILES else "_full"
plt.savefig(f'images/loss_plot{filename_suffix}.png', dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
# 3. Confusion Matrix
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Environment', 'Gunshot'], 
            yticklabels=['Environment', 'Gunshot'],
            cbar_kws={'label': 'Number of Samples'})
plt.xlabel('Predicted Label', fontsize=14)
plt.ylabel('True Label', fontsize=14)
plt.tight_layout()
# Fix the filename formatting
filename_suffix = f"_{NUM_GUNSHOT_FILES}" if NUM_GUNSHOT_FILES else "_full"
plt.savefig(f'images/confusion_matrix{filename_suffix}.png', dpi=600, bbox_inches='tight')
plt.show()

In [ ]:

# ========== DETAILED METRICS ==========
print("\n" + "="*60)
print("DETAILED PERFORMANCE METRICS")
print("="*60)
print(classification_report(y_test, y_pred, target_names=['Environment', 'Gunshot']))

# Calculate additional metrics
tn, fp, fn, tp = cm.ravel()
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2 * (precision * recall) / (precision + recall)
specificity = tn / (tn + fp)

print(f"\nAdditional Metrics:")
print(f"Precision: {precision:.4f}")
print(f"Recall (Sensitivity): {recall:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"F1-Score: {f1_score:.4f}")
print(f"Test Accuracy: {acc:.4f}")

print(f"\nConfusion Matrix Breakdown:")
print(f"True Positives (Correctly identified gunshots): {tp}")
print(f"True Negatives (Correctly identified environment sounds): {tn}")
print(f"False Positives (Environment sounds classified as gunshots): {fp}")
print(f"False Negatives (Gunshots classified as environment sounds): {fn}")

print(f"\nPlots saved:")
print("   - accuracy_plot.png & accuracy_plot.pdf")
print("   - loss_plot.png & loss_plot.pdf")
print("   - confusion_matrix.png & confusion_matrix.pdf")

In [ ]:
# ========== EXPERIMENT TRACKING ==========
print("\n" + "EXPERIMENT SUMMARY" + "="*45)
print(f"Dataset Size: {NUM_ENVIRONMENT_FILES if NUM_ENVIRONMENT_FILES else 'ALL'} env + {NUM_GUNSHOT_FILES if NUM_GUNSHOT_FILES else 'ALL'} gunshot files")
print(f"Total Samples Used: {len(X)}")
print(f"Test Accuracy: {acc:.4f} ({acc*100:.2f}%)")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1_score:.4f}")
print("="*65)

# Save results to compare different dataset sizes
experiment_results = {
    'dataset_size': len(X),
    'env_files': NUM_ENVIRONMENT_FILES if NUM_ENVIRONMENT_FILES else 'ALL',
    'gun_files': NUM_GUNSHOT_FILES if NUM_GUNSHOT_FILES else 'ALL',
    'test_accuracy': acc,
    'precision': precision,
    'recall': recall,
    'f1_score': f1_score,
    'true_positives': tp,
    'true_negatives': tn,
    'false_positives': fp,
    'false_negatives': fn
}

print(f"\nCopy this result for comparison:")
print(f"Dataset: {experiment_results['env_files']}/{experiment_results['gun_files']} | Acc: {acc:.3f} | F1: {f1_score:.3f} | Precision: {precision:.3f} | Recall: {recall:.3f}")

In [ ]:
# ========== DATA EXPORT ==========

import json
from datetime import datetime

def export_data():
    """Export all study data to meet journal requirements"""
    
    # Create export directory
    export_dir = "data_export"
    if not os.path.exists(export_dir):
        os.makedirs(export_dir)
        os.makedirs(f"{export_dir}/metrics")
        os.makedirs(f"{export_dir}/figures_data")
        os.makedirs(f"{export_dir}/raw_data")
    
    print("Exporting data for journal requirements...")
    
    # 1. Export training history (epoch-by-epoch values)
    training_history_df = pd.DataFrame({
        'epoch': range(0, len(train_acc)),
        'training_accuracy': train_acc,
        'validation_accuracy': val_acc,
        'training_loss': train_loss,
        'validation_loss': val_loss
    })
    training_history_df.to_csv(f"{export_dir}/metrics/training_history_per_epoch_{filename_suffix}.csv", index=False)
    
    # 2. Export figure data points (exact values used in graphs)
    # Accuracy plot data
    accuracy_plot_data = pd.DataFrame({
        'epoch': range(0, len(train_acc)),
        'training_accuracy': train_acc,
        'validation_accuracy': val_acc
    })
    accuracy_plot_data.to_csv(f"{export_dir}/figures_data/accuracy_plot_data_{filename_suffix}.csv", index=False)
    
    # Loss plot data  
    loss_plot_data = pd.DataFrame({
        'epoch': range(0, len(train_loss)),
        'training_loss': train_loss,
        'validation_loss': val_loss
    })
    loss_plot_data.to_csv(f"{export_dir}/figures_data/loss_plot_data_{filename_suffix}.csv", index=False)
    
    # Confusion matrix data
    confusion_matrix_data = pd.DataFrame(cm, 
                                       index=['True_Environment', 'True_Gunshot'],
                                       columns=['Predicted_Environment', 'Predicted_Gunshot'])
    confusion_matrix_data.to_csv(f"{export_dir}/figures_data/confusion_matrix_data_{filename_suffix}.csv")
    
    # 4. Export detailed metrics and statistics
    detailed_metrics = {
        'export_timestamp': datetime.now().isoformat(),
        'dataset_info': {
            'total_samples': len(X),
            'environment_samples': NUM_ENVIRONMENT_FILES if NUM_ENVIRONMENT_FILES else 'ALL',
            'gunshot_samples': NUM_GUNSHOT_FILES if NUM_GUNSHOT_FILES else 'ALL',
            'test_samples': len(y_test),
            'feature_shape': list(X[0].shape)
        },
        'model_performance': {
            'test_accuracy': float(acc),
            'test_loss': float(loss),
            'precision': float(precision),
            'recall': float(recall),
            'f1_score': float(f1_score),
            'specificity': float(specificity)
        },
        'confusion_matrix_breakdown': {
            'true_positives': int(tp),
            'true_negatives': int(tn),
            'false_positives': int(fp),
            'false_negatives': int(fn)
        },
        'training_parameters': {
            'epochs': 10,
            'batch_size': 8,
            'validation_split': 0.1,
            'optimizer': 'adam',
            'loss_function': 'binary_crossentropy',
            'random_state': 42
        }
    }
    
    with open(f"{export_dir}/metrics/complete_study_metrics_{filename_suffix}.json", 'w') as f:
        json.dump(detailed_metrics, f, indent=2)
    
    # 5. Export means, standard deviations, and statistical measures
    statistical_summary = pd.DataFrame({
        'Metric': ['Training Accuracy', 'Validation Accuracy', 'Training Loss', 'Validation Loss'],
        'Mean': [np.mean(train_acc), np.mean(val_acc), np.mean(train_loss), np.mean(val_loss)],
        'Std_Deviation': [np.std(train_acc), np.std(val_acc), np.std(train_loss), np.std(val_loss)],
        'Min_Value': [np.min(train_acc), np.min(val_acc), np.min(train_loss), np.min(val_loss)],
        'Max_Value': [np.max(train_acc), np.max(val_acc), np.max(train_loss), np.max(val_loss)],
        'Final_Value': [train_acc[-1], val_acc[-1], train_loss[-1], val_loss[-1]]
    })
    statistical_summary.to_csv(f"{export_dir}/metrics/statistical_summary_{filename_suffix}.csv", index=False)
    
    print(f"All data exported to: {export_dir}/")
    print(f"Files created:")
    for root, dirs, files in os.walk(export_dir):
        for file in files:
            rel_path = os.path.relpath(os.path.join(root, file), export_dir)
            print(f"   - {rel_path}")
    
    return export_dir

# Import pandas for data export
import pandas as pd

# Run the export
export_directory = export_data()
print(f"\ndata export completed!")

In [ ]:
# ========== MODEL EXPORT ==========
import tensorflow as tf
import os
from datetime import datetime

# Quick model export function
def quick_export_model():
    """Export model in key formats for deployment"""
    
    # Create export directory
    models_dir = "exported_models"
    os.makedirs(f"{models_dir}/keras", exist_ok=True)
    model_name = f"gunshot_detection"
    
    results = {}
    
    print("🚀 Exporting model formats...")
    
    # Keras native format
    try:
        keras_path = f"{models_dir}/keras/{model_name}.keras" 
        model.save(keras_path)
        results['keras'] = keras_path
        print(f" Keras format: {keras_path}")
    except Exception as e:
        print(f" Keras failed: {e}")
    
    return results

# Export models
print("Starting model export...")
export_results = quick_export_model()

print(f"\nExport completed! Available formats:")
for format_name, path in export_results.items():
    if isinstance(path, str) and not any(x in format_name for x in ['size', 'percent']):
        print(f" {format_name.upper()}: {os.path.basename(path)}")

# Show file sizes
if 'tflite_size_kb' in export_results:
    print(f"\nModel sizes:")
    print(f"   TensorFlow Lite: {export_results['tflite_size_kb']:.1f} KB")
    if 'quantized_size_kb' in export_results:
        print(f"   Quantized TFLite: {export_results['quantized_size_kb']:.1f} KB") 
        print(f"   Space saved: {export_results['compression_percent']:.1f}%")

print(f"\nAll files saved in: exported_models/")
print("\nModel export and quantization complete!")

In [ ]:
# ========== FULL INT8 QUANTIZATION FOR MICROCONTROLLERS ==========
# Create fully quantized INT8 model for RP2040 and other microcontrollers

def representative_dataset():
    """Representative dataset for calibrating quantization"""
    for i in range(100):
        yield [X_train[i:i+1].astype(np.float32)]

print("Converting to fully quantized INT8 TFLite model...")
print("This model will be optimized for microcontrollers like RP2040")

try:
    # Create converter with full integer quantization
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = representative_dataset
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.inference_input_type = tf.int8  
    converter.inference_output_type = tf.int8
    
    # Convert the model
    tflite_model_int8 = converter.convert()
    
    # Save the quantized model
    models_dir = "exported_models"
    os.makedirs(f"{models_dir}/quantized", exist_ok=True)
    quantized_filename = "exported_models/quantized/gunshot_model_quant.tflite"
    with open(quantized_filename, "wb") as f:
        f.write(tflite_model_int8)
    
    # Get file size info
    int8_size_kb = len(tflite_model_int8) / 1024
    
    print(f"\nSaved: {quantized_filename}")
    print(f"INT8 Model size: {int8_size_kb:.1f} KB")

    print(f"\nTo convert to C header file for RP2040:")
    print(f"   xxd -i {quantized_filename} > gunshot_model_quant.h")

    print(f"\nMicrocontroller deployment notes:")
    print(f"   - Input/Output: INT8 format (quantized)")
    print(f"   - Input shape: [1, 44, 13, 1]")
    print(f"   - Memory requirement: ~{int8_size_kb:.1f} KB for model")
    
except Exception as e:
    print(f"INT8 quantization failed: {e}")
    print("This might happen if the model is not suitable for full integer quantization")

print("\nFull INT8 quantization complete!")